In [18]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from scipy.optimize import approx_fprime
from pbi_proximity_measure import pbi_kktpm
from apbi_proximity_measure import apbi_kktpm


# 1) Define the ZDT1 problem restricted to 2 variables
class ZDT1_2D(Problem):
    """
    ZDT1 problem restricted to 2 variables:
        Min f1 = x[0]
        Min f2 = (1 + 9*x[1]) * (1 - sqrt(x[0] / (1 + 9*x[1])))
        s.t. 0 <= x[0] <= 1, 0 <= x[1] <= 1
    """
    def __init__(self):
        super().__init__(
            n_var=2, 
            n_obj=2, 
            n_constr=0, 
            xl=np.array([0.0, 0.0]), 
            xu=np.array([1.0, 1.0])
        )

    def _evaluate(self, x, out, *args, **kwargs):
        # x shape is (n, 2)
        f1 = x[:, 0]
        g  = 1.0 + 9.0 * x[:, 1]
        f2 = g * (1 - np.sqrt(x[:, 0] / g))
        out["F"] = np.column_stack([f1, f2])

        g1 =   -x[:, 0] 
        g2 =   -x[:, 1] 
        g3 = +x[:, 0] - 1
        g4 = +x[:, 1] - 1

        out["G"] = np.column_stack([g1, g2, g3, g4])


def d1(x, w, r_star):
    X_batch = np.array([x])
    out = {}
    problem._evaluate(X_batch, out)
    F_x = out["F"][0]  # shape (2,) for 2 objectives
    w = np.array(w)
    w = w / np.linalg.norm(w)
    r_star = np.array(r_star)
    F_x = np.array(F_x)
    # d1: projection of (F(x) - r_star) onto w
    d1 = np.dot(F_x - r_star, w) / np.linalg.norm(w)
    return d1
    
def d2(x, w, r_star):
    X_batch = np.array([x])
    out = {}
    problem._evaluate(X_batch, out)
    F_x = out["F"][0]  # shape (2,) for 2 objectives
    w = np.array(w)
    w = w / np.linalg.norm(w)
    r_star = np.array(r_star)
    F_x = np.array(F_x)
    # d2: perpendicular distance from F(x) to the line L
    d1 = np.dot(F_x - r_star, w) / np.linalg.norm(w)
    beta = 10  # Higher values make approximation closer to the original norm
    diff_sq = (F_x - (r_star + (d1 * w)))**2
    d2_smooth = (1 / beta) * np.log(np.sum(np.exp(beta * diff_sq)))
    return d2_smooth
    
def evaluate_constraints(x, problem):
    """
    Evaluate constraints for a single solution x (which is 2D: shape (1, n_var)).
    Returns
    -------
    np.ndarray
        1D array of combined inequality constraints [g(x), h(x), -h(x)] if they exist.
        Empty array if no constraints exist.
    """
    out = {}
    problem._evaluate(x, out)
    # If out has "G", store it; else create empty array of shape (1, 0)
    g_arr = np.array(out["G"]) if "G" in out else np.empty((x.shape[0], 0))
    # If out has "H", store it; else create empty array
    h_arr = np.array(out["H"]) if "H" in out else np.empty((x.shape[0], 0))
    # Transform equality constraints H(x) = 0 into two inequalities: H(x) <= 0 and -H(x) <= 0
    if h_arr.size > 0:
        transformed_h = np.column_stack([h_arr, -h_arr])
    else:
        transformed_h = np.empty((x.shape[0], 0))
    # Combine G and transformed H
    if g_arr.size > 0 and transformed_h.size > 0:
        combined_constraints = np.column_stack([g_arr, transformed_h])
    elif g_arr.size > 0:
        combined_constraints = g_arr
    else:
        combined_constraints = transformed_h
    # If constraints exist, return the constraints of the first row; otherwise return empty
    if combined_constraints.size > 0:
        return combined_constraints[0]
    else:

        return np.empty((0,))
    
def single_constraint_func(x, idx, problem):
    """
    Evaluate the idx-th constraint for a single solution x. x is 1D list and problem is a pymoo problem instance.
    """
    # Make sure x is 2D for problem._evaluate
    X2D = x.reshape(1, -1)
    return evaluate_constraints(X2D, problem)[idx]



# 2) Define the PBI measure function:
#    PBI(x, w, r*, sigma) = d1 + sigma * d2
#    with d1 = ((F(x) - r*)^T w) / ||w||
#         d2 = || F(x) - (r* + d1 w) ||
def pbi_measure(x, problem, w, sigma, r_star):
    """
    x: 1D numpy array of decision variables (e.g., shape (2,))
    problem: a pymoo problem instance to evaluate F(x)
    w: weight vector
    sigma: the sigma parameter
    r_star: reference point (array-like)
    """
    # Evaluate the objectives F(x). For a single point, we can do this manually:
    # We'll create a batch of size 1 to pass into the problem.
    X_batch = np.array([x])
    out = {}
    problem._evaluate(X_batch, out)
    F_val = out["F"][0]  # shape (2,) for 2 objectives

    # Compute d1
    diff = (F_val - r_star)  # e.g., shape (2,)
    w_norm = np.linalg.norm(w)
    d1 = abs(np.dot(diff, w) / w_norm)

    # Compute d2
    delta = 1
    d2_vec = diff - d1 * w  # shape (2,)
    d2 = np.linalg.norm(d2_vec)
    d2_smooth_pseudo_huber = np.sum(delta**2 * (np.sqrt(1.0 + (d2_vec / delta)**2) - 1.0))

    pbi_value = d1 + sigma * d2_smooth_pseudo_huber
    return pbi_value


X_batch = np.array([[0.347935, 0.0]])
out = {}
problem = ZDT1_2D()
problem._evaluate(X_batch, out)
F_val = out["F"][0]  # shape (2,) for 2 objectives

# 4) Instantiate the problem and define the needed parameters
zdt1_problem = ZDT1_2D()                 # Our custom 2D ZDT1 problem
w_special = np.array([0.46, 0.54])       # Special weight
sigma_val = 5.0                          # Sigma = 5
r_star = np.array([0,0])            # Reference point

# We create a small "phi_func" closure that uses the above parameters:
def phi_func_zdt1(x):
    return pbi_measure(x, zdt1_problem, w_special, sigma_val, r_star)

g_functions = []
for i in range(4):
    # Use default argument trick: i=i prevents late binding in lambda
    g_functions.append(lambda zz, i=i: single_constraint_func(zz, i, problem))

# 5) Evaluate the six sample points from Table 9
points_zdt1 = {
    "A": np.array([0.347935, 0.000000]),
    "B": np.array([0.360000, 0.003835]),
    "C": np.array([0.450000, 0.032580]),
    "D": np.array([0.520000, 0.054950]),
    "E": np.array([0.600000, 0.080500]),
    "F": np.array([0.760000, 0.131580]),
}
results = []
print("ZDT1 (2D) APBI-KKTPM Tests\n---------------------------")
for label, x_val in points_zdt1.items():
    epsilon_A = apbi_kktpm(x_val, phi_func_zdt1, g_functions, implementation="paper_version")
    epsilon_A_updated = apbi_kktpm(x_val, phi_func_zdt1, g_functions, implementation="complicate_linear_system")
    if epsilon_A is None:
        epsilon_A = -1
    if epsilon_A_updated is None:
        epsionm_A_updated = -1

    epsilon_pbi = pbi_kktpm(x_val, phi_func_zdt1, g_functions)

    # Calculate the gradients of d1 and d2
    grad_d1 = approx_fprime(x_val, lambda x: d1(x,w_special, r_star), epsilon=1e-6)
    grad_d2 = approx_fprime(x_val, lambda x: d2(x,w_special, r_star), epsilon=1e-6)


    X_batch = np.array([x_val])
    out = {}
    zdt1_problem._evaluate(X_batch, out)
    f_val = out["F"][0]
    f1, f2 = f_val[0], f_val[1]

    
    results.append((label, x_val,f1, f2, epsilon_A, epsilon_A_updated, epsilon_pbi))
    


for label, x_val,f1, f2, epsilon_A, epsilon_A_updated, epsilon_pbi in results:
    print(f"Point {label} -> x={x_val}, f1 = {f1}, f2 = {f2} APBI-KKTPM={epsilon_A} APBI-KKTPM Updated={epsilon_A_updated} EPBI-KKTPM={epsilon_pbi}")


ZDT1 (2D) APBI-KKTPM Tests
---------------------------
The solver could not find an optimal solution.
Failed to solve the linear system. We now try with quadratic system.
The shape of GGT is:  (4, 4)
The value of A1A1T is:  [[35.28650783]]
The value of A1BJT is:  [[ 2.19329188e-03 -5.94024436e+00 -2.19329188e-03  5.94024436e+00]]
The value of BJA1T is:  [[ 2.19329188e-03]
 [-5.94024436e+00]
 [-2.19329188e-03]
 [ 5.94024436e+00]]
The value of BJBJT is:  [[ 1.  0. -1.  0.]
 [ 0.  1.  0. -1.]
 [-1.  0.  1.  0.]
 [ 0. -1.  0.  1.]]
The value of GGT is:  [[0.12105876 0.         0.22687624 0.347935  ]
 [0.         0.         0.         0.        ]
 [0.22687624 0.         0.42518876 0.652065  ]
 [0.347935   0.         0.652065   1.        ]]
The shape of LHS is:  (5, 5)
The value of S is:  [0]
The value of LHS is:  [[ 3.62865078e+01  2.19329188e-03 -5.94024436e+00 -2.19329188e-03
   5.94024436e+00]
 [ 2.19329188e-03  1.12105876e+00  0.00000000e+00 -7.73123764e-01
   3.47935000e-01]
 [-5.94024

In [19]:

# This is the case for the SRN problem


class CustomSRN(Problem):

    def __init__(self):
        super().__init__(n_var=2, n_obj=2, n_constr=2, xl=np.array([-20, -20]), xu=np.array([20, 20]))

    def _evaluate(self, x, out, *args, **kwargs):
        # Objective functions
        f1 = (x[:, 0] - 2) ** 2 + (x[:, 1] - 1) ** 2 + 2
        f2 = 9 * x[:, 0] - (x[:, 1] - 1) ** 2

        # Constraints
        g1 = -225 + x[:, 0] ** 2 + x[:, 1] ** 2  # g1(x) <= 0
        g2 = (x[:, 0] - 3 * x[:, 1]) + 10 # g2(x) <= 0

        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1, g2])  # Constraints



X_batch = np.array([[0.347935, 0.0]])
out = {}
problem = CustomSRN() # Our custom 2D ZDT1 problem
problem._evaluate(X_batch, out)
F_val = out["F"][0]  # shape (2,) for 2 objectives
print("F_val2", F_val)

# 4) Instantiate the problem and define the needed parameters          
w_special = np.array([0.5, 0.5])       # Special weight
sigma_val = 5.0                          # Sigma = 5
r_star = np.array([0, -250])            # Reference point

# We create a small "phi_func" closure that uses the above parameters:
def phi_func(x):
    return pbi_measure(x, problem, w_special, sigma_val, r_star)

g_functions = []
for i in range(problem.n_constr):
    # Use default argument trick: i=i prevents late binding in lambda
    g_functions.append(lambda zz, i=i: single_constraint_func(zz, i, problem))

# 5) Evaluate the six sample points from Table 9
points_srn = {
    "A": np.array([-2.5, 11.130391]),
    "B": np.array([-3.0, 10.899495]),
    "C": np.array([-4.791288, 9.909556]),
    "D": np.array([-8.0, 7.1644140]),
    "E": np.array([-9.588723, 4.701552]),
    "F": np.array([7.0, 12.958261]),
}
results = []
print("SRN APBI-KKTPM Tests\n---------------------------")
for label, x_val in points_srn.items():
    epsilon_A = apbi_kktpm(x_val, phi_func_zdt1, g_functions, implementation="paper_version")
    epsilon_A_updated = apbi_kktpm(x_val, phi_func_zdt1, g_functions, implementation="complicate_linear_system")
    if epsilon_A is None:
        epsilon_A = -1
    if epsilon_A_updated is None:
        epsionm_A_updated = -1

    epsilon_pbi = pbi_kktpm(x_val, phi_func_zdt1, g_functions)

    # Calculate the gradients of d1 and d2
    grad_d1 = approx_fprime(x_val, lambda x: d1(x,w_special, r_star), epsilon=1e-6)
    grad_d2 = approx_fprime(x_val, lambda x: d2(x,w_special, r_star), epsilon=1e-6)




    X_batch = np.array([x_val])
    out = {}
    problem._evaluate(X_batch, out)
    f_val = out["F"][0]
    f1, f2 = f_val[0], f_val[1]

    
    results.append((label, x_val,f1, f2, epsilon_A, epsilon_A_updated, epsilon_pbi))
    


for label, x_val,f1, f2, epsilon_A, epsilon_A_updated, epsilon_pbi in results:
    print(f"Point {label} -> x={x_val}, f1 = {f1}, f2 = {f2} APBI-KKTPM={epsilon_A} APBI-KKTPM Updated={epsilon_A_updated} EPBI-KKTPM={epsilon_pbi}")


F_val2 [5.72931876 2.131415  ]
SRN APBI-KKTPM Tests
---------------------------
The solver could not find an optimal solution.
Failed to solve the linear system. We now try with quadratic system.
The shape of GGT is:  (2, 2)
The value of A1A1T is:  [[nan]]
The value of A1BJT is:  [[nan nan]]
The value of BJA1T is:  [[nan]
 [nan]]
The value of BJBJT is:  [[520.54244972 -71.78234802]
 [-71.78234802  10.00000001]]
The value of GGT is:  [[8999.25366395 2456.15049322]
 [2456.15049322  670.35283932]]
The shape of LHS is:  (3, 3)
The value of S is:  [0]
The value of LHS is:  [[          nan           nan           nan]
 [          nan 9519.79611367 2384.3681452 ]
 [          nan 2384.3681452   680.35283932]]
The value of RHS is:  [1. 0. 0.]
The solver could not find an optimal solution.
The shape of GGT is:  (2, 2)
The value of A1A1T is:  [[nan]]
The value of A1BJT is:  [[nan nan]]
The value of BJA1T is:  [[nan]
 [nan]]
The value of BJBJT is:  [[520.54244972 -71.78234802]
 [-71.78234802  10.0

C:\Users\Lukag\AppData\Local\Temp\ipykernel_11960\3557757851.py:30: RuntimeWarning: invalid value encountered in sqrt
  f2 = g * (1 - np.sqrt(x[:, 0] / g))
